In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc

import plotnine as p9

import liana as li
import decoupler as dc
import omnipath as op
import matplotlib
import os

In [ ]:
from matplotlib import rcParams

custom_params = {"axes.spines.right": False, "axes.spines.top": False}
rcParams['font.family'] = 'sans-serif'
matplotlib.rcParams['font.sans-serif'] = ['Arial']
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
import seaborn as sns
sns.set_theme(style="ticks", rc=custom_params)
import importlib
importlib.reload(li)
# verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.verbosity = 3
sc.logging.print_header()
mycmap = matplotlib.colors.LinearSegmentedColormap.from_list('mycmap', ["#004369", "white", "#DB1F48"])
sc.set_figure_params(dpi=900, color_map="bwr")
%matplotlib inline


In [3]:
os.chdir('/mnt/data/hong/2022/DHJ1_human_obesity_placenta/')
adata = sc.read_h5ad('output/10x_h5/h5ad/ad_v2.h5ad')

In [4]:
naga = adata[adata.obs['group']=='Normal_AGA']

In [5]:
naga.X = naga.layers['log_norm'].copy()

In [6]:
dea_df = pd.read_csv('output/communication/all_de_ccc_input.tsv', sep='\t')

In [13]:
dea_df_lr = dea_df[~dea_df['final_celltypes'].isin(['B cells', 'EVT', 'eSTB', 'pCTB', 'Endo-2', 'Myofibroblast', 'T cells', 'NK cells', 'B cells', 'Monocyte'])]

In [14]:
dea_df_lr = dea_df_lr.set_index('Unnamed: 0')

In [16]:
lr_res = li.multi.df_to_lr(naga,
                           dea_df=dea_df_lr,
                           resource_name='consensus',
                           expr_prop=0, # calculated for adata as passed - used to filter interactions
                           groupby='final_celltypes',
                           stat_keys=['score', 'p'],
                           use_raw=False,
                           complex_col='score', 
                           verbose=True,
                           return_all_lrs=False,
                           )

Using resource `consensus`.
Using `.X`!
1307 features of mat are empty, they will be removed.
/mnt/data/hong/anaconda3/envs/scanpy/lib/python3.10/site-packages/liana/method/_pipe_utils/_pre.py:148: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
0.81 of entities in the resource are missing from the data.


In [18]:
lr_res.to_csv('output/communication/all_de_intravillous_lr_major.tsv', sep='\t')